In [8]:
# Importing pre-trained models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19

# Core Keras packages
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Other packages
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split


In [9]:
def load_image_data(directory):
    
    images = []
    labels = []
    for label in sorted(os.listdir(directory)):
        for image in os.listdir(directory + "/" + label ):
            img = cv2.resize(cv2.imread(directory + "/" + label+ "/" +image), (224, 224))
            images.append(img)
            labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return (images,labels)

In [10]:
x_test,y_test=load_image_data(r'C:\Users\Tharun B S\Documents\VS Code\real-time-sign-language-translator\augmented')

In [11]:
len(x_test)

450

In [12]:
x_test=x_test.astype(float)/255.0

In [13]:
x_test1= np.expand_dims(x_test, axis = 1)

In [14]:
x_test1[0].shape

(1, 224, 224, 3)

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_test_cat = le.fit_transform(y_test)
y_test_cat = to_categorical(y_test_cat)
y_test_cat[0]

array([1., 0., 0.])

In [17]:
model= keras.models.load_model(r"C:\Users\Tharun B S\Documents\VS Code\real-time-sign-language-translator\sign_language_model.h5")

In [18]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import os

In [19]:
labels=list(np.unique(y_test))

In [ ]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

 
offset = 20
imgSize = 224
  
labels = ['Hello', 'Please', 'Sorry', ]
 
while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
 
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
 
        imgCropShape = imgCrop.shape
 
        aspectRatio = h / w
 
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            imgWhite1 = cv2.resize(imgWhite, (300, 300))  # Resize to training size
            imgWhite1 = imgWhite1 / 255.0                 # Normalize (important!)
            imgWhite1 = imgWhite1.reshape(1, 300, 300, 3) # Reshape for prediction

            prediction = model.predict(imgWhite1)
            index = np.argmax(prediction)
            print(prediction, index)

 
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            imgWhite1 = cv2.resize(imgWhite, (300, 300))  # Resize to training size
            imgWhite1 = imgWhite1 / 255.0                 # Normalize (important!)
            imgWhite1 = imgWhite1.reshape(1, 300, 300, 3) # Reshape for prediction

            prediction = model.predict(imgWhite1)
            index = np.argmax(prediction)
            print(prediction, index)

 
 
        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y - 26), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 2)

        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)
 
 
        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)
 
    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
[[0.997389   0.00132716 0.00128391]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
[[0.99558395 0.00156338 0.00285265]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
[[9.912612e-01 7.524099e-04 7.986453e-03]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
[[9.9259609e-01 1.3842764e-04 7.2654439e-03]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
[[0.92701787 0.02351042 0.04947179]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
[[0.2632997  0.01611573 0.7205846 ]] 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
[[0.00217642 0.9744747  0.02334886]] 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
[[0.96295345 0.01384061 0.02320593]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
[[0.98440844 0.00550826 0.01008339]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
[[9.9979657e-01 7.5413845e-05 1.2800073e-04]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
[[9.99786317e-01 1.02107755e-04 1.11606780e-04]] 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
[[9.9979919e-01 1.0157373e-04 9.9270372e-05]] 0
1/1 ━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

: 